In [1]:
import polars as pl
import numpy as np
from utils import CDNOW_sample, rfm_summary
from scipy.special import gammaln, hyp2f1, gamma
from scipy.optimize import minimize

import altair as alt
from IPython.display import display_markdown

Source:
- ["Counting Your Customers" the Easy Way: An Alternative to the Pareto/NBD Model](https://www.brucehardie.com/abstracts/abstract-fhl_2004-04.html)
- [Implementing the BG/NBD Model for Customer Base Analysis in Excel](https://www.brucehardie.com/notes/004/)
- [A Note on Implementing the Pareto/NBD Model in MATLAB](https://www.brucehardie.com/notes/008/)

In [2]:
CDNOW = CDNOW_sample()

calwk = 273 # 39 week calibration period

rfm_data = rfm_summary(CDNOW, calwk)

# Time of trial purchase (in weeks)
tofp = (
    CDNOW
    .filter(pl.col('DoR') == 0)
    .with_columns((pl.col('PurchDay') / 7).alias('Time of First Purch'))
    .group_by('Time of First Purch').agg(pl.len().alias('Count'))
    .sort('Time of First Purch')
)

# actual weekly & cumulative repeat sales data
actual_repeat_sales = (
    CDNOW
    .filter(pl.col('DoR') != 0)
    .with_columns(((pl.col('PurchDay') - 1) // 7 + 1).alias('Week'))
    .group_by('Week')
    .agg(pl.len().cast(pl.Float32).alias('Weekly Sales'))
    .sort('Week')
    .with_columns(pl.col('Weekly Sales').cum_sum().cast(pl.Float32).alias('Cum Sales'))
    .collect()    
)

actual_repeat_sales = pl.concat([pl.DataFrame({'Week': 1, 'Weekly Sales': 0, 'Cum Sales': 0}, 
                                              schema={'Week': pl.UInt16, 'Weekly Sales': pl.Float32, 'Cum Sales': pl.Float32}), 
                                 actual_repeat_sales])

forecast_horizon_week = (calwk * 2) // 7
forecast_horizon_day = forecast_horizon_week * 7
forecast_horizon_day
t = np.arange(forecast_horizon_day, dtype=np.int16) + 1

tofp_array = tofp.collect().to_numpy()
num_triers = tofp_array[:, 1]
trial_weeks = tofp_array[:, 0]
trial_days = np.arange(np.max(trial_weeks)*7, dtype=np.int16) + 1

In [3]:
def bgnbd_est(rfm_data, guess={'r': 0.01, 'alpha': 0.01, 'a': 0.01, 'b':0.01}):
    
    def log_likelihood(x):
        r, alpha, a, b = x
        p1x, t_x, T  = rfm_data[:,0], rfm_data[:,1], rfm_data[:,2]

        ln_A_1 = gammaln(p1x + r) - gammaln(r) + r * np.log(alpha)
        ln_A_2 = gammaln(a + b) + gammaln(b + p1x) - gammaln(b) - gammaln(a + b + p1x)
        ln_A_3 = -(r + p1x) * np.log(alpha + T)
        ln_A_4 = np.where(p1x > 0, 
                          np.log(a) - np.log(b + p1x - 1) - (r + p1x) * np.log(alpha + t_x),
                          0)
        return -np.sum(ln_A_1 + ln_A_2 + np.log(np.exp(ln_A_3) + (p1x > 0) * np.exp(ln_A_4)))
    
    bnds = [(0, np.inf) for _ in range(4)]
    return minimize(log_likelihood, x0=list(guess.values()), bounds=bnds, method='Nelder-Mead', options={'maxiter':10000})

result = bgnbd_est(rfm_data.select('P1X', 't_x', 'T').collect().to_numpy())
r, alpha, a, b = result.x
ll = result.fun

# Sample Parameters
# r = 0.24259395230803
# alpha = 4.41359091416604
# a = 0.792919955839573
# b = 2.42589404751842

display_markdown(f'''$r$ = {r:0.4f}

$\\alpha$ = {alpha:0.4f}

$a$ = {a:0.4f}

$b$ = {b:0.4f}

Log-Likelihood = {-ll:0.4f}''', raw=True)

/var/folders/0s/z9xp988n3j78zfjwg3y616x00000gn/T/ipykernel_9755/3800996231.py:11: RuntimeWarning: invalid value encountered in log
  np.log(a) - np.log(b + p1x - 1) - (r + p1x) * np.log(alpha + t_x),
/var/folders/0s/z9xp988n3j78zfjwg3y616x00000gn/T/ipykernel_9755/3800996231.py:11: RuntimeWarning: divide by zero encountered in log
  np.log(a) - np.log(b + p1x - 1) - (r + p1x) * np.log(alpha + t_x),
/var/folders/0s/z9xp988n3j78zfjwg3y616x00000gn/T/ipykernel_9755/3800996231.py:8: RuntimeWarning: invalid value encountered in subtract
  ln_A_2 = gammaln(a + b) + gammaln(b + p1x) - gammaln(b) - gammaln(a + b + p1x)
/var/folders/0s/z9xp988n3j78zfjwg3y616x00000gn/T/ipykernel_9755/3800996231.py:11: RuntimeWarning: invalid value encountered in subtract
  np.log(a) - np.log(b + p1x - 1) - (r + p1x) * np.log(alpha + t_x),
/var/folders/0s/z9xp988n3j78zfjwg3y616x00000gn/T/ipykernel_9755/3800996231.py:13: RuntimeWarning: invalid value encountered in add
  return -np.sum(ln_A_1 + ln_A_2 + np.log(np.ex

$r$ = 0.2426

$\alpha$ = 4.4136

$a$ = 0.7929

$b$ = 2.4259

Log-Likelihood = -9582.4292

In [4]:
z = (t/7) / (alpha + (t/7))
h2f1 = hyp2f1(r, b, (a + b - 1), z)
E_X_t = (a + b - 1) / (a - 1) * (1 - (alpha / (alpha + (t/7)))**r * h2f1)

index = t.reshape(-1, 1) - trial_days
index = np.clip(index-1, 0, E_X_t.shape[0]).T

# Compute cumulative repeat sales
cum_rpt_sls = np.dot(num_triers, np.triu(E_X_t[index], k=1))[6::7]

# Compute weekly repeat sales
wkly_rpt_sls = np.diff(cum_rpt_sls, prepend=0)

bgnbd_repeat_sales = pl.DataFrame({'Week': np.arange(cum_rpt_sls.shape[0]) + 1, 
                                       'Weekly Sales': wkly_rpt_sls,
                                       'Cum Sales': cum_rpt_sls}, 
                                      schema={'Week': pl.UInt16, 'Weekly Sales': pl.Float32 ,'Cum Sales': pl.Float32})

#### Computing Conditional Expectations

In [5]:
x = 2
t_x = 30.43
T = 38.86
t_cust = 39 # the length of the period over which we wish to make the conditional forecast

a_cust = r+x
b_cust = b+x
c_cust = a+b+x-1
z_cust = t_cust/(alpha + T + t_cust)

h2f1_cust = hyp2f1(a_cust, b_cust, c_cust, z_cust)

E_Y_X = (a + b + x - 1) / (a - 1) * (1 - ((alpha + T) / (alpha + T + t_cust))**(r + x) * h2f1_cust) / \
        (1 + (x > 0) * a / (b + x - 1) * ((alpha + T) / (alpha + t_x))**(r + x))

display_markdown(f'''$E(Y(t) \\mid X = x, t_{{x}}, T, r, \\alpha, a, b)$ = {E_Y_X:0.4f}''' ,raw=True)

$E(Y(t) \mid X = x, t_{x}, T, r, \alpha, a, b)$ = 1.2259

### Pareto/NBD Model

In [6]:
def paretonbd_est(rfm_data, guess={'r': 1, 'alpha': 1, 's': 1, 'beta': 1}):
    
    def log_likelihood(x):
        r, alpha, s, beta = x
        p1x, t_x, T = rfm_data[:,0], rfm_data[:,1], rfm_data[:,2]
        
        maxab = np.max((alpha, beta))
        absab = np.abs(alpha - beta)
        param2 = s + 1
        if alpha < beta:
            param2 = r + p1x
            
        part1 = (alpha**r * beta**s / gamma(r)) * gamma(r+p1x)
        part2 = 1/((alpha+T)**(r+p1x) * (beta+T)**s)

        if absab == 0:
            F1 = 1/((maxab+t_x)**(r+s+p1x))
            F2 = 1/((maxab+T)**(r+s+p1x))
        else:
            F1 = hyp2f1(r+s+p1x, param2, r+s+p1x+1, absab/(maxab+t_x)) / ((maxab+t_x)**(r+s+p1x))
            F2 = hyp2f1(r+s+p1x, param2, r+s+p1x+1, absab/(maxab+T)) / ((maxab+T)**(r+s+p1x))        
        
        return -np.sum(np.log(part1*(part2+(s/(r+s+p1x))*(F1-F2))))
    
    bnds = [(1e-6, 20) for _ in range(4)]
    return minimize(log_likelihood, x0=list(guess.values()), bounds=bnds, method='Nelder-Mead', options={'maxiter':10000})

result = paretonbd_est(rfm_data.select('P1X', 't_x', 'T').collect().to_numpy())
r, alpha, s, beta = result.x
ll = result.fun

# Sample Parameters
# r = 0.553268332737686
# alpha = 10.577643207793674
# s = 0.606338658139013
# beta = 11.672047422351444

display_markdown(f'''$r$ = {r:0.4f}

$\\alpha$ = {alpha:0.4f}

$s$ = {a:0.4f}

$\\beta$ = {b:0.4f}

Log-Likelihood = {-ll:0.4f}''', raw=True)

$r$ = 0.5533

$\alpha$ = 10.5777

$s$ = 0.7929

$\beta$ = 2.4259

Log-Likelihood = -9594.9762

In [7]:
E_X_t = r*beta/(alpha*(s-1)) * (1 - (beta/(beta + (t/7)))**(s-1))

index = t.reshape(-1, 1) - trial_days
index = np.clip(index-1, 0, E_X_t.shape[0]).T

# Compute cumulative repeat sales
cum_rpt_sls = np.dot(num_triers, np.triu(E_X_t[index], k=1))[6::7]

# # Compute weekly repeat sales
wkly_rpt_sls = np.diff(cum_rpt_sls, prepend=0)

paretonbd_repeat_sales = pl.DataFrame({'Week': np.arange(cum_rpt_sls.shape[0]) + 1, 
                                       'Weekly Sales': wkly_rpt_sls,
                                       'Cum Sales': cum_rpt_sls}, 
                                      schema={'Week': pl.UInt16, 'Weekly Sales': pl.Float32 ,'Cum Sales': pl.Float32})

In [8]:
cum_sales_plot_data = (
    actual_repeat_sales
    .rename({'Cum Sales': 'Actual'})
    .join(other=paretonbd_repeat_sales, on='Week', how='left', suffix=' - Pareto/NBD')
    .rename({'Cum Sales': 'Pareto/NBD'})
    .join(other=bgnbd_repeat_sales, on='Week', how='left', suffix=' - BG/NBD')
    .rename({'Cum Sales': 'BG/NBD'})
    .unpivot(on=['Actual', 'Pareto/NBD', 'BG/NBD'], index='Week', value_name='Cum Sales', variable_name='Actual Vs Estimated')
)

(
    alt.Chart(cum_sales_plot_data).mark_line()
    .encode(x=alt.X('Week', title='Week'), 
            y=alt.Y('Cum Sales', title='Cum. Rpt Transactions'), 
            strokeDash='Actual Vs Estimated:N')
    .properties(
            width=550,
            height=400,
            title='Tracking Cumulative Repeat Transactions'
        ).configure_view(stroke=None).configure_axisY(grid=False).configure_axisX(grid=False)         
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [9]:
weekly_sales_plot_data = (
    actual_repeat_sales
    .rename({'Weekly Sales': 'Actual'})
    .join(other=paretonbd_repeat_sales, on='Week', how='left', suffix=' - Pareto/NBD')
    .rename({'Weekly Sales': 'Pareto/NBD'})
    .join(other=bgnbd_repeat_sales, on='Week', how='left', suffix=' - BG/NBD')
    .rename({'Weekly Sales': 'BG/NBD'})
    .unpivot(on=['Actual', 'Pareto/NBD', 'BG/NBD'], index='Week', value_name='Weekly Sales', variable_name='Actual Vs Estimated')
)

(
    alt.Chart(weekly_sales_plot_data).mark_line()
    .encode(x=alt.X('Week', title='Week'), 
            y=alt.Y('Weekly Sales', title='Weekly Rpt Transactions'), 
            strokeDash='Actual Vs Estimated:N')
    .properties(
            width=650,
            height=250,
            title='Tracking Weekly Repeat Transactions'
        ).configure_view(stroke=None).configure_axisY(grid=False).configure_axisX(grid=False)         
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [10]:
# Compute P(active|p1x,tx,T)
rfm_data_array = rfm_data.select('P1X', 'P2X', 't_x', 'T').collect().to_numpy()

p1x = rfm_data_array[:,0].astype(np.int16)
p2x = rfm_data_array[:,1].astype(np.int16)
t_x = rfm_data_array[:,2]
T = rfm_data_array[:,3]

maxab = np.max((alpha, beta))
absab = np.abs(alpha - beta)
param2 = s + 1
if alpha < beta:
    param2 = r + p1x

F0 = (alpha + T)**(r + p1x) * (beta + T)**s

if absab == 0:
    F1 = 1/((maxab+t_x)**(r+s+p1x))
    F2 = 1/((maxab+T)**(r+s+p1x))
else:
    F1 = hyp2f1(r+s+p1x, param2, r+s+p1x+1, absab/(maxab+t_x)) / ((maxab+t_x)**(r+s+p1x))
    F2 = hyp2f1(r+s+p1x, param2, r+s+p1x+1, absab/(maxab+T)) / ((maxab+T)**(r+s+p1x))       

pactive = 1 / (1 + (s / (r+s+p1x)) * F0 * (F1-F2))

# compute average P(active|p1x,tx,T) and determine the proportion of customers buying in the second 39 weeks for each level of p1x
# Compute unique levels of p1x
unique_p1x = np.arange(p1x.max() + 1)

# Compute counts per unique p1x
np1x = np.bincount(p1x)

# Compute actual proportions of active customers
pa_actual = np.divide(np.bincount(p1x, weights=(p2x > 0).astype(float)), np1x, where=np1x!=0)

# Compute estimated proportions of active customers
pa_est = np.divide(np.bincount(p1x, weights=pactive), np1x, where=np1x!=0)

In [11]:
# create right-censored version for plot
censor = 7  # Right-censor at 7+
denom = np.sum(np1x[censor:]) # Compute denominator

# Compute censored actual proportions
pa_act_cen = np.zeros(censor + 1)
pa_act_cen[:censor] = pa_actual[:censor]
pa_act_cen[censor] = np.dot(np1x[censor:], pa_actual[censor:]) / denom

# Compute censored estimated proportions
pa_est_cen = np.zeros(censor + 1)
pa_est_cen[:censor] = pa_est[:censor]
pa_est_cen[censor] = np.dot(np1x[censor:], pa_est[censor:]) / denom

prop_active_customers = pl.DataFrame({'P1X': [str(i) if i < 7 else '7+' for i in range(8)], 'Empirical': pa_act_cen, 'Pareto/NBD': pa_est_cen})
prop_active_customers = prop_active_customers.unpivot(on=['Empirical', 'Pareto/NBD'], index='P1X', variable_name='Actual Vs Estimated', value_name='P(active)')

(
    alt.Chart(prop_active_customers).mark_line()
    .encode(x=alt.X('P1X:O', title='# Transactions in Weeks 1−39', axis=alt.Axis(labelAngle=0)), 
            y=alt.Y('P(active):Q', title='P(Active)'), 
            strokeDash='Actual Vs Estimated:N')
    .properties(
            width=550,
            height=400,
            title='Predicted vs. Actual Proportions of Active Customers'
        ).configure_view(stroke=None).configure_axisY(grid=False).configure_axisX(grid=False)         
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [17]:
def bgnbd_est(rfm_data, guess={'r': 0.01, 'alpha': 0.01, 'a': 0.01, 'b':0.01}):
    
    def log_likelihood(x):
        r, alpha, a, b = x
        p1x, t_x, T  = rfm_data[:,0], rfm_data[:,1]/4, rfm_data[:,2]/4 # change the unit of time in the above example from week to quad-week

        ln_A_1 = gammaln(p1x + r) - gammaln(r) + r * np.log(alpha)
        ln_A_2 = gammaln(a + b) + gammaln(b + p1x) - gammaln(b) - gammaln(a + b + p1x)
        ln_A_3 = -(r + p1x) * np.log(alpha + T)
        ln_A_4 = np.where(p1x > 0, 
                          np.log(a) - np.log(b + p1x - 1) - (r + p1x) * np.log(alpha + t_x),
                          0)
        return -np.sum(ln_A_1 + ln_A_2 + np.log(np.exp(ln_A_3) + (p1x > 0) * np.exp(ln_A_4)))
    
    bnds = [(0, np.inf) for _ in range(4)]
    return minimize(log_likelihood, x0=list(guess.values()), bounds=bnds, method='Nelder-Mead', options={'maxiter':10000})

result = bgnbd_est(rfm_data.select('P1X', 't_x', 'T').collect().to_numpy())
r, alpha, a, b = result.x
ll = result.fun

display_markdown(f'''$r$ = {r:0.4f}

$\\alpha$ = {alpha:0.4f}

$a$ = {a:0.4f}

$b$ = {b:0.4f}

Log-Likelihood = {-ll:0.4f}''', raw=True)

/var/folders/0s/z9xp988n3j78zfjwg3y616x00000gn/T/ipykernel_9755/1538701586.py:11: RuntimeWarning: invalid value encountered in log
  np.log(a) - np.log(b + p1x - 1) - (r + p1x) * np.log(alpha + t_x),
/var/folders/0s/z9xp988n3j78zfjwg3y616x00000gn/T/ipykernel_9755/1538701586.py:7: RuntimeWarning: divide by zero encountered in log
  ln_A_1 = gammaln(p1x + r) - gammaln(r) + r * np.log(alpha)
/var/folders/0s/z9xp988n3j78zfjwg3y616x00000gn/T/ipykernel_9755/1538701586.py:11: RuntimeWarning: divide by zero encountered in log
  np.log(a) - np.log(b + p1x - 1) - (r + p1x) * np.log(alpha + t_x),
/var/folders/0s/z9xp988n3j78zfjwg3y616x00000gn/T/ipykernel_9755/1538701586.py:8: RuntimeWarning: invalid value encountered in subtract
  ln_A_2 = gammaln(a + b) + gammaln(b + p1x) - gammaln(b) - gammaln(a + b + p1x)
/var/folders/0s/z9xp988n3j78zfjwg3y616x00000gn/T/ipykernel_9755/1538701586.py:13: RuntimeWarning: invalid value encountered in add
  return -np.sum(ln_A_1 + ln_A_2 + np.log(np.exp(ln_A_3) + (

$r$ = 0.2426

$\alpha$ = 1.1034

$a$ = 0.7929

$b$ = 2.4259

Log-Likelihood = -6176.3040

In [25]:
def bgnbd_est(rfm_data, guess={'r': 0.01, 'alpha': 0.01, 'a': 0.01, 'b':0.01}):
    
    def log_likelihood(x):
        r, alpha, a, b = x
        p1x, t_x, T  = rfm_data[:,0], rfm_data[:,1], rfm_data[:,2]

        D_1 = gammaln(r + p1x) - gammaln(r) + gammaln(a + b) + gammaln(b + p1x) - gammaln(b) - gammaln(a + b + p1x)
        D_2 = r * np.log(alpha) - (r + p1x) * np.log(alpha + t_x)
        C_3 = ((alpha + t_x) / (alpha + T))**(r + p1x)
        C_4 = a / (b + p1x - 1)
        
        return -np.sum(D_1 + D_2 + np.where(p1x > 0, np.log(C_3 + C_4), C_3))
    
    bnds = [(0, np.inf) for _ in range(4)]
    return minimize(log_likelihood, x0=list(guess.values()), bounds=bnds, method='Nelder-Mead', options={'maxiter':10000})

result = bgnbd_est(rfm_data.select('P1X', 't_x', 'T').collect().to_numpy())
r, alpha, a, b = result.x
ll = result.fun

display_markdown(f'''$r$ = {r:0.4f}

$\\alpha$ = {alpha:0.4f}

$a$ = {a:0.4f}

$b$ = {b:0.4f}

Log-Likelihood = {-ll:0.4f}''', raw=True)

/var/folders/0s/z9xp988n3j78zfjwg3y616x00000gn/T/ipykernel_9755/1133703615.py:8: RuntimeWarning: divide by zero encountered in log
  D_2 = r * np.log(alpha) - (r + p1x) * np.log(alpha + t_x)
/var/folders/0s/z9xp988n3j78zfjwg3y616x00000gn/T/ipykernel_9755/1133703615.py:8: RuntimeWarning: invalid value encountered in subtract
  D_2 = r * np.log(alpha) - (r + p1x) * np.log(alpha + t_x)
/var/folders/0s/z9xp988n3j78zfjwg3y616x00000gn/T/ipykernel_9755/1133703615.py:12: RuntimeWarning: divide by zero encountered in log
  return -np.sum(D_1 + D_2 + np.where(p1x > 0, np.log(C_3 + C_4), C_3))
/var/folders/0s/z9xp988n3j78zfjwg3y616x00000gn/T/ipykernel_9755/1133703615.py:7: RuntimeWarning: invalid value encountered in subtract
  D_1 = gammaln(r + p1x) - gammaln(r) + gammaln(a + b) + gammaln(b + p1x) - gammaln(b) - gammaln(a + b + p1x)
/var/folders/0s/z9xp988n3j78zfjwg3y616x00000gn/T/ipykernel_9755/1133703615.py:10: RuntimeWarning: divide by zero encountered in divide
  C_4 = a / (b + p1x - 1)
/var

$r$ = 0.6070

$\alpha$ = 6.6628

$a$ = 0.7206

$b$ = 1.8625

Log-Likelihood = -7907.7903

In [23]:
# Sample Parameters
r = 0.24259395230803
alpha = 4.41359091416604
a = 0.792919955839573
b = 2.42589404751842

p1x, t_x, T  = 200, 38, 40

ln_A_1 = gammaln(p1x + r) - gammaln(r) + r * np.log(alpha)
ln_A_2 = gammaln(a + b) + gammaln(b + p1x) - gammaln(b) - gammaln(a + b + p1x)
ln_A_3 = -(r + p1x) * np.log(alpha + T)
ln_A_4 = np.where(p1x > 0, 
                    np.log(a) - np.log(b + p1x - 1) - (r + p1x) * np.log(alpha + t_x),
                    0)
ll = -np.sum(ln_A_1 + ln_A_2 + np.log(np.exp(ln_A_3) + (p1x > 0) * np.exp(ln_A_4)))
    
ll

/var/folders/0s/z9xp988n3j78zfjwg3y616x00000gn/T/ipykernel_9755/1413133053.py:15: RuntimeWarning: divide by zero encountered in log
  ll = -np.sum(ln_A_1 + ln_A_2 + np.log(np.exp(ln_A_3) + (p1x > 0) * np.exp(ln_A_4)))


np.float64(inf)